In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import glob
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import datetime
from sqlalchemy import create_engine
from psycopg2.extras import execute_values
import psycopg2
import sqlalchemy
%load_ext sql

## Inspection

### Iterate through CSVs, Concat Files, and Create DataFrame:

In [5]:
files = glob.glob('data/JC-2016*-citibike-tripdata.csv')
df_list = []
for filename in files: 
    data = pd.read_csv(filename)
    df_list.append(data)
df = pd.concat(df_list)    

### Inspect the DataFrame:

Columns: Index, Trip Duration, Start Time, Stop Time, Start Station ID, Start Station Name, Start Station Latitude, Start Station Longitude
End Station ID, End Station Name, End Station Latitude, End Station Longitude, Bike ID, User Type, Birth Year, Gender

In [9]:
df.head(5)

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,361,2016-02-01 00:31:18,2016-02-01 00:37:19,3202,Newport PATH,40.727224,-74.033759,3203,Hamilton Park,40.727596,-74.044247,24393,Subscriber,1975.0,1
1,297,2016-02-01 01:55:05,2016-02-01 02:00:02,3195,Sip Ave,40.730743,-74.063784,3194,McGinley Square,40.725340,-74.067622,24394,Subscriber,1985.0,2
2,1155,2016-02-01 02:40:05,2016-02-01 02:59:20,3183,Exchange Place,40.716247,-74.033459,3210,Pershing Field,40.742677,-74.051789,24676,Subscriber,1976.0,1
3,1769,2016-02-01 05:11:28,2016-02-01 05:40:58,3214,Essex Light Rail,40.712774,-74.036486,3203,Hamilton Park,40.727596,-74.044247,24700,Subscriber,1974.0,2
4,935,2016-02-01 05:48:24,2016-02-01 06:03:59,3203,Hamilton Park,40.727596,-74.044247,3214,Essex Light Rail,40.712774,-74.036486,24639,Subscriber,1974.0,2


In [11]:
#Start Time, Stop Time, and Birth Year seem like they should be timestamps and dates, but are objects. 
df.dtypes

Trip Duration                int64
Start Time                  object
Stop Time                   object
Start Station ID             int64
Start Station Name          object
Start Station Latitude     float64
Start Station Longitude    float64
End Station ID               int64
End Station Name            object
End Station Latitude       float64
End Station Longitude      float64
Bike ID                      int64
User Type                   object
Birth Year                 float64
Gender                       int64
dtype: object

In [13]:
#[247584 rows x 15 columns]
df.describe

<bound method NDFrame.describe of        Trip Duration           Start Time            Stop Time  \
0                361  2016-02-01 00:31:18  2016-02-01 00:37:19   
1                297  2016-02-01 01:55:05  2016-02-01 02:00:02   
2               1155  2016-02-01 02:40:05  2016-02-01 02:59:20   
3               1769  2016-02-01 05:11:28  2016-02-01 05:40:58   
4                935  2016-02-01 05:48:24  2016-02-01 06:03:59   
...              ...                  ...                  ...   
19483            249  2016-05-31 23:16:00  2016-05-31 23:20:10   
19484            650  2016-05-31 23:18:32  2016-05-31 23:29:23   
19485           2048  2016-05-31 23:25:28  2016-05-31 23:59:36   
19486            455  2016-05-31 23:31:57  2016-05-31 23:39:32   
19487            239  2016-05-31 23:47:38  2016-05-31 23:51:37   

       Start Station ID Start Station Name  Start Station Latitude  \
0                  3202       Newport PATH               40.727224   
1                  3195          

In [15]:
#User Type has 380 null values, <br> and Birth Year has 18,999 null values
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 247584 entries, 0 to 19487
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Trip Duration            247584 non-null  int64  
 1   Start Time               247584 non-null  object 
 2   Stop Time                247584 non-null  object 
 3   Start Station ID         247584 non-null  int64  
 4   Start Station Name       247584 non-null  object 
 5   Start Station Latitude   247584 non-null  float64
 6   Start Station Longitude  247584 non-null  float64
 7   End Station ID           247584 non-null  int64  
 8   End Station Name         247584 non-null  object 
 9   End Station Latitude     247584 non-null  float64
 10  End Station Longitude    247584 non-null  float64
 11  Bike ID                  247584 non-null  int64  
 12  User Type                247204 non-null  object 
 13  Birth Year               228585 non-null  float64
 14  Gender    

In [17]:
df.shape

(247584, 15)

### Locating the Column With the Most Unique Locations

In [20]:
#Are there more starts or ends? 51 starts
start_station_names = df['Start Station Name'].unique()
start_station_names.shape

(51,)

In [22]:
#There are more ends: 102 ends
end_station_names = df['End Station Name'].unique()
end_station_names.shape

(102,)

## Cleaning

### Changing DataTypes

In [26]:
df['Trip Duration'] = pd.to_timedelta(df['Trip Duration'], unit='s')
df['Start Time'] = pd.to_datetime(df['Start Time'])
df['Stop Time'] = pd.to_datetime(df['Stop Time'])

### New 'Date' Column

In [29]:
df['Date'] = pd.to_datetime(df['Start Time']).dt.date

### Missing Values

### Addressing NaN User Types

In [33]:
user_type = df['User Type'].unique()
user_type

array(['Subscriber', 'Customer', nan], dtype=object)

### Multiple Imputation for Birth Year NaN

In [36]:
#Providing a test series for trialing the Imputation
birth_year_test = {'X': [1975.0, 1985.0, 1976.0, 1974.0, 1974.0, np.nan, 1990.0, 1988.0, 1984.0, 1986.0]}

In [38]:
#Creating the DataFrame for the test data
df_test = pd.DataFrame(data = birth_year_test)

In [40]:
#Setting up the imputer
imp = IterativeImputer(max_iter=10, random_state=0)

In [42]:
imp.fit(df_test)

IterativeImputer(random_state=0)

In [44]:
# Applying Imputation to DataFrame
df['Birth Year'] = np.round(imp.fit_transform(df[['Birth Year']]), 1)

In [46]:
# New Column 'Age'
df['Age'] = 2016 - df['Birth Year']

### Dropping the Few Remaining NaN Rows

In [49]:
df.dropna(inplace=True)

### Haversine Formula Calculates New Column 'Trip Distance' 

In [54]:
#Earth radius in kilometers (use 3958.8 for miles)
R = 6371  

# Convert latitude and longitude from degrees to radians
df['start_lat_rad'] = np.radians(df['Start Station Latitude'])
df['end_lat_rad'] = np.radians(df['End Station Latitude'])
df['start_lon_rad'] = np.radians(df['Start Station Longitude'])
df['end_lon_rad'] = np.radians(df['End Station Longitude'])

# Differences in coordinates
df['delta_lat'] = df['end_lat_rad'] - df['start_lat_rad']
df['delta_lon'] = df['end_lon_rad'] - df['start_lon_rad']

# Haversine formula
a = np.sin(df['delta_lat'] / 2)**2 + np.cos(df['start_lat_rad']) * np.cos(df['end_lat_rad']) * np.sin(df['delta_lon'] / 2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

# Compute trip distance
df['Trip Length (km)'] = R * c

# Drop intermediate columns if needed
df.drop(columns=['start_lat_rad', 'end_lat_rad', 'start_lon_rad', 'end_lon_rad', 'delta_lat', 'delta_lon'], inplace=True)

print(df[['Start Station Latitude', 'End Station Latitude', 'Start Station Longitude', 'End Station Longitude', 'Trip Length (km)']])


       Start Station Latitude  End Station Latitude  Start Station Longitude  \
0                   40.727224             40.727596               -74.033759   
1                   40.730743             40.725340               -74.063784   
2                   40.716247             40.742677               -74.033459   
3                   40.712774             40.727596               -74.036486   
4                   40.727596             40.712774               -74.044247   
...                       ...                   ...                      ...   
19483               40.719586             40.724176               -74.043117   
19484               40.724176             40.721525               -74.050656   
19485               40.728745             40.728745               -74.032108   
19486               40.734961             40.746730               -74.059503   
19487               40.717732             40.721525               -74.043845   

       End Station Longitude  Trip Leng

### Create Alternate Frame With Only Unique Values in the End Station ID column

In [57]:
#Since the DB table 'locations needs only unique values for IDs
unique_df = df.groupby('End Station ID', as_index=False).first()

### Read-In & Inspect the Weather CSV

In [60]:
nw = pd.read_csv('data/newark_airport_2016.csv')
#16 columns: STATION, NAME, DATE, AWND, PTGM, PRCP, SNOW, SNWD, TAVG, TMAX, TMIN, TSUN, WDF2, WDF5, WSF2, WSF5 
nw.head()

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
0,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-01,12.75,NaN,0.0,0.0,0.0,41,43,34,NaN,270,280.0,25.9,35.1
1,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-02,9.40,NaN,0.0,0.0,0.0,36,42,30,NaN,260,260.0,21.0,25.1
2,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-03,10.29,NaN,0.0,0.0,0.0,37,47,28,NaN,270,250.0,23.9,30.0
3,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-04,17.22,NaN,0.0,0.0,0.0,32,35,14,NaN,330,330.0,25.9,33.1
4,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-05,9.84,NaN,0.0,0.0,0.0,19,31,10,NaN,360,350.0,25.1,31.1


In [62]:
#365 rows
nw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   STATION  366 non-null    object 
 1   NAME     366 non-null    object 
 2   DATE     366 non-null    object 
 3   AWND     366 non-null    float64
 4   PGTM     0 non-null      float64
 5   PRCP     366 non-null    float64
 6   SNOW     366 non-null    float64
 7   SNWD     366 non-null    float64
 8   TAVG     366 non-null    int64  
 9   TMAX     366 non-null    int64  
 10  TMIN     366 non-null    int64  
 11  TSUN     0 non-null      float64
 12  WDF2     366 non-null    int64  
 13  WDF5     364 non-null    float64
 14  WSF2     366 non-null    float64
 15  WSF5     364 non-null    float64
dtypes: float64(9), int64(4), object(3)
memory usage: 45.9+ KB


### Cleaning the Weather Frame

In [65]:
# Don't need the completely empty columns
nw.drop(['PGTM', 'TSUN'], axis=1, inplace=True)

In [67]:
# Changing the data types
nw['DATE'] = pd.to_datetime(nw['DATE'])

In [69]:
#Drop a few incomplete rows
nw = nw.dropna()

## Database

### Setting Up SQL DB

### Queries for SQL DB in Postbird Postgresql

### Establish DB Connection

In [98]:
DB_USER = 'lydiakonstanski'
DB_PASSWORD = 'Celeste'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = "Citibike_db"

# Create the database engine
engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')
# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
)
cur = conn.cursor()

### Filling Postbird DB

In [100]:
trips_tuples2 = [tuple(row) for row in df[['User Type']].dropna().to_numpy()]
trips_insert2 = 'INSERT INTO trips (user_type) VALUES %s'
execute_values(cur, trips_insert2, trips_tuples2)
conn.commit()
cur.close()
conn.close()

## Creating Views

Questions to answer: 
* Time on the bike: Max, Min, Average, Distribution
* Distance on the bike: Max, Min, Average, Distribution 
* Temps: Max, Min, Average, Distribution

* Count of trips by gender
* Length of trip by gender
* Count of trips by age
* Length of trip by age
* Avg temp to avg bike trip
* Count of trips in a temp range (with gender, with age)
* Count of trips with snow (with gender, with age)
* Count of trips with wind (with gender, with age)
* Count of trips with sun (with gender, with age)
* Avg time of trips in a temp range
* Count of trips started in each temp range
* Length of trips with snow
* Length of trips with wind speed > avg wind speed
* Length of trips with wind speed < avg wind speed
* Length of trips with sun
* When do non-subscribers use bikes the most? At any certain station?

Columns: 
df
Trip Duration
Start Station Latitude
Start Station Longitude
End Station Latitude
End Station Longitude
Gender
Birth Year
NEW COLUMN: Trip Length

nw 
TAVG
TMAX
TMIN
SNWD
AWND


In [ ]:
# time_on_bike view to find: Max, Min, Average, Distribution
cur.execute(CREATE VIEW time_on_bike AS t SELECT trips

In [ ]:
# Distance on the bike view to find: Max, Min, Average, Distribution 
cur.execute(CREATE VIEW distance_on_bike AS d SELECT trips

In [ ]:
# temps by month view to find: Months, Avg_Temp

In [ ]:
# count of trips by gender: count of trips where Users.gender = 1,  count of trips where Users.gender = 0

In [ ]:
# len of trips by gender: count of trips where Users.gender = 1,  count of trips where Users.gender = 0